# Prompt engineering to improve results

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kluster-ai/klusterai-cookbook/blob/main/examples/prompt-engineering.ipynb)

## Config

Enter your personal kluster.ai API key (make sure it has no blank spaces). Remember to <a href="https://platform.kluster.ai/signup" target="_blank">sign up</a> if you don't have one yet.

In [1]:
from getpass import getpass
# Enter you personal kluster.ai API key (make sure in advance it has no blank spaces)
api_key = getpass("Enter your kluster.ai API key: ")

Enter your kluster.ai API key:  ········


## Setup

In [2]:
%pip install -q OpenAI

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import urllib.request
import pandas as pd
import numpy as np
import random
import requests
from openai import OpenAI
import time
import json
from IPython.display import clear_output, display

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000, 'display.max_colwidth', 500)

In [4]:
# Set up the client
client = OpenAI(
    base_url="https://api.kluster.ai/v1",
    api_key=api_key,
)

## 1. Perform a classification task

In [5]:
# IMDB Top 1000 dataset:
url = "https://raw.githubusercontent.com/kluster-ai/klusterai-cookbook/refs/heads/main/data/imdb_top_1000.csv"
urllib.request.urlretrieve(url,filename='imdb_top_1000.csv')

# Load and process the dataset based on URL content
df = pd.read_csv('imdb_top_1000.csv', usecols=['Series_Title', 'Overview', 'Genre'])
df.head(3)

,Series_Title,Genre,Overview
0,The Shawshank Redemption,Drama,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency."
1,The Godfather,"Crime, Drama",An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.
2,The Dark Knight,"Action, Crime, Drama","When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice."


#### Create the Batch File

In [6]:
def create_tasks(df, task_type, system_prompt, model):
    tasks = []
    for index, row in df.iterrows():
        content = row['Overview']
        
        task = {
            "custom_id": f"{task_type}-{index}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": model,
                "temperature": 0.5,
                "response_format": {"type": "json_object"},
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": content},
                ],
            }
        }
        tasks.append(task)
    return tasks

def save_tasks(tasks, task_type):
    filename = f"batch_tasks_{task_type}.jsonl"
    with open(filename, 'w') as file:
        for task in tasks:
            file.write(json.dumps(task) + '\n')
    return filename

In [7]:
ASSISTANT_PROMPT = '''
    You are a helpful assitant that classifies movie genres based on the movie description. Choose one of the following options: 
    Action, Adventure, Animation, Biography, Comedy, Crime, Drama, Family, Fantasy, Film-Noir, History, Horror, Music, Musical, Mystery, Romance, Sci-Fi, Sport, Thriller, War, Western.
    Provide your response as a single word with the matching genre.
    '''

task_list = create_tasks(df, system_prompt=ASSISTANT_PROMPT, model="klusterai/Meta-Llama-3.1-8B-Instruct-Turbo", task_type='assistant')
filename = save_tasks(task_list, task_type='assistant')

#### Upload Batch File to kluster.ai

In [8]:
def create_batch_job(file_name):
    print(f"Creating batch job for {file_name}")
    batch_file = client.files.create(
        file=open(file_name, "rb"),
        purpose="batch"
    )

    batch_job = client.batches.create(
        input_file_id=batch_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )

    return batch_job

job = create_batch_job(filename)

Creating batch job for batch_tasks_assistant.jsonl


#### Check Job progress

In [9]:
def parse_json_objects(data_string):
    if isinstance(data_string, bytes):
        data_string = data_string.decode('utf-8')

    json_strings = data_string.strip().split('\n')
    json_objects = []

    for json_str in json_strings:
        try:
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")

    return json_objects

def monitor_job_status(client, job_id, task_type):
    all_completed = False

    while not all_completed:
        all_completed = True
        output_lines = []

        updated_job = client.batches.retrieve(job_id)

        if updated_job.status.lower() != "completed":
            all_completed = False
            completed = updated_job.request_counts.completed
            total = updated_job.request_counts.total
            output_lines.append(f"{task_type.capitalize()} job status: {updated_job.status} - Progress: {completed}/{total}")
        else:
            output_lines.append(f"{task_type.capitalize()} job completed!")

        # Clear the output and display updated status
        clear_output(wait=True)
        for line in output_lines:
            display(line)

        if not all_completed:
            time.sleep(10)

monitor_job_status(client=client, job_id=job.id, task_type='assistant')

'Assistant job completed!'

#### Get the results

In [10]:
batch_job = client.batches.retrieve(job.id)
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content
results = parse_json_objects(result)
answers = []

for res in results:
    task_id = res['custom_id']
    result = res['response']['body']['choices'][0]['message']['content']
    answers.append(result) 

df['answer'] = answers

In [11]:
# Step 1: Create a column with True/False if answer is in Genre
df['is_correct'] = df.apply(
    lambda row: row['answer'] in row['Genre'].split(', '),
    axis=1
)

# Step 2: Calculate accuracy as the mean of the 'is_correct' column
accuracy = df['is_correct'].mean()
accuracy

0.6

In [12]:
df[df.is_correct==False].answer.value_counts()

answer
Drama.                        33
Fantasy                       33
Western.                      33
Thriller.                     31
Thriller                      30
Fantasy.                      24
Romance.                      17
Biography                     17
Crime.                        17
Romance                       16
Drama                         16
Comedy.                       13
Adventure.                     9
Sci-Fi                         8
Biography.                     7
Film-Noir                      7
Mystery.                       6
Action.                        6
Mystery                        5
Action                         5
Comedy                         5
Western                        4
Film-Noir.                     4
War                            4
Musical                        4
History                        3
Family.                        3
History.                       3
Horror                         3
Family                         3
Mus

## 2. Prompt Engineering

Now that we identified what are kind of mistakes the LLM is doing, we'll modify the original prompt to help it perform better

In [13]:
ASSISTANT_PROMPT = '''
    Instruction:
    You are an assistant that classifies a movie into exactly one of the listed genres based on a provided movie description.
    
    Process to Follow (Do Not Reveal to User):
     1.	Read the movie description carefully.
     2.	Consider each of the following genres and compare the details of the description to the typical features of each genre.
     3.	Select the single best-matching genre. If you feel uncertain, re-check the features and do not guess outside the provided genres.
     4.	Once certain, provide your final answer as instructed below.
    
    Available Genres:
    Action, Adventure, Animation, Biography, Comedy, Crime, Drama, Family, Fantasy, Film-Noir, History, Horror, Music, Musical, Mystery, Romance, Sci-Fi, Sport, Thriller, War, Western
    
    Output Requirements:
    - Output exactly one word (the chosen genre) without any additional punctuation or explanation.
    - Do not invent new genres or deviate from the provided list.
    
    Final Answer Format:
    GenreName
    '''

task_list = create_tasks(df, task_type='assistant', system_prompt=ASSISTANT_PROMPT, model="klusterai/Meta-Llama-3.1-8B-Instruct-Turbo")
filename = save_tasks(task_list, task_type='assistant')

#### Upload Batch File to kluster.ai

In [14]:
job = create_batch_job(filename)

Creating batch job for batch_tasks_assistant.jsonl


#### Check job progress

In [15]:
monitor_job_status(client=client, job_id=job.id, task_type='assistant')

'Assistant job completed!'

#### Get the results

In [16]:
batch_job = client.batches.retrieve(job.id)
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content
results = parse_json_objects(result)
answers = []

for res in results:
    task_id = res['custom_id']
    result = res['response']['body']['choices'][0]['message']['content']
    answers.append(result) 

df['improved_answer'] = answers

In [17]:
# Step 1: Create a column with True/False if answer is in Genre
df['improved_is_correct'] = df.apply(
    lambda row: row['improved_answer'] in row['Genre'].split(', '),
    axis=1
)

# Step 2: Calculate accuracy as the mean of the 'is_correct' column
accuracy = df['improved_is_correct'].mean()
accuracy

0.717